In [1]:
import os
import sys
sep = os.sep
import numpy as np
from testarch.unet import runs as ru
from testarch.unet import runs as rm
import utils.auto_split as asp

Total Params: 31042434


In [2]:
from PIL import Image as IMG
from utils.img_utils import Image
from sklearn.metrics import precision_recall_fscore_support
from scipy.ndimage.morphology import distance_transform_edt

In [3]:
runs_unet = [
    ru.DRIVE_1_100_1, ru.DRIVE_1_1, ru.DRIVE_WEIGHTED,
    ru.STARE_1_100_1, ru.STARE_1_1, ru.STARE_WEIGHTED,
    ru.WIDE_1_100_1, ru.WIDE_1_1, ru.WIDE_WEIGHTED,
    ru.CHASEDB_1_100_1, ru.CHASEDB_1_1, ru.CHASEDB_WEIGHTED,
    ru.VEVIO_MOSAICS_1_100_1, ru.VEVIO_MOSAICS_1_1, ru.VEVIO_MOSAICS_WEIGHTED,
    ru.VEVIO_FRAMES_1_100_1, ru.VEVIO_FRAMES_1_1, ru.VEVIO_FRAMES_WEIGHTED]

runs_miniunet = [
    rm.DRIVE_1_100_1, rm.DRIVE_1_1, rm.DRIVE_WEIGHTED,
    rm.STARE_1_100_1, rm.STARE_1_1, rm.STARE_WEIGHTED,
    rm.WIDE_1_100_1, rm.WIDE_1_1, rm.WIDE_WEIGHTED,
    rm.CHASEDB_1_100_1, rm.CHASEDB_1_1, rm.CHASEDB_WEIGHTED,
    rm.VEVIO_MOSAICS_1_100_1, rm.VEVIO_MOSAICS_1_1, rm.VEVIO_MOSAICS_WEIGHTED,
    rm.VEVIO_FRAMES_1_100_1, rm.VEVIO_FRAMES_1_1, rm.VEVIO_FRAMES_WEIGHTED]

In [4]:
log_file = 'UNET_F1_WEIGHTED.csv'

In [5]:
with open(log_file, 'w+') as log:
    log.write('DATASET,EXPERIMENT,PREC_AVG,RECALL_AVG,F1')
    for d in runs_unet:
        dset = d['Dirs']['logs'].split(os.sep)[1]
        exp = d['Dirs']['logs'].split(os.sep)[-1]
        results = {'p':[],'r':[], 'f1':[]}

        test_files = []
        for split_file in os.listdir(d['Dirs']['splits_json']):
            test_files += asp.load_split_json(d['Dirs']['splits_json']+sep+split_file)['test']

        for file in test_files:
            image = Image()
            image.load_file(d['Dirs']['image'], file)
            image.load_ground_truth(d['Dirs']['truth'], d['Funcs']['truth_getter'])

            if d['Dirs'].get('mask'):
                image.load_mask(d['Dirs']['mask'], d['Funcs']['mask_getter'])

            pred_file =  'pred_' + file.split('.')[0] + '.png'
            image.pred = np.array(IMG.open(d['Dirs']['logs'] + sep + pred_file))

            image.pred[image.pred==255] = 1

            gt = image.ground_truth.copy()
            gt[gt==255] = 1
            edt_gt = distance_transform_edt(gt)

            edt = edt_gt.copy()
            edt[edt==0] = 1
            edt = 1/edt

            p, r, f1, _ = precision_recall_fscore_support(gt.flatten(), image.pred.flatten(),\
                 sample_weight=edt.flatten(), average='binary')
            results['p'].append(p)
            results['r'].append(r)
            results['f1'].append(f1)
        p_avg = np.array(results['p']).mean()
        r_avg = np.array(results['r']).mean()
        f1 =  (2 * p_avg * r_avg) / (p_avg + r_avg)
        line = '{},{},{},{},{}'.format(dset, exp, round(p_avg, 5), round(r_avg, 5), round(f1, 5))
        log.write('\n'+line)
        log.flush()

### SPLIT FOUND:  data\DRIVE\splits\UNET-DRIVE.json Loaded
### SPLIT FOUND:  data\DRIVE\splits\UNET-DRIVE.json Loaded
### SPLIT FOUND:  data\DRIVE\splits\UNET-DRIVE.json Loaded
### SPLIT FOUND:  data\STARE\splits\0UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\12UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\16UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\4UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\8UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\0UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\12UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\16UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\4UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\8UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\0UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\12UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\16UNET-STARE.json Loaded
### SPLIT FOUND:  data\STARE\splits\4

In [6]:
re

NameError: name 're' is not defined